In [2]:
import pandas as pd
#import ydata_profiling as yp
# data preprocessing
from sklearn.preprocessing import StandardScaler
# data splitting
from sklearn.model_selection import train_test_split
from collections import Counter
from sklearn import neighbors
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix,accuracy_score,roc_curve,classification_report,matthews_corrcoef,roc_auc_score
import seaborn as sns
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

In [4]:
data = pd.read_csv("/home/cloud/Datasets/Hotel Reservations_data_features.csv")

In [6]:
data.head(10)

,no_of_adults,no_of_children,no_of_weekend_nights,no_of_week_nights,required_car_parking_space,lead_time,repeated_guest,avg_price_per_room,no_of_previous_bookings_not_canceled,no_of_special_requests,...,arrival_month_3,arrival_month_4,arrival_month_5,arrival_month_6,arrival_month_7,arrival_month_8,arrival_month_9,arrival_month_10,arrival_month_11,arrival_month_12
0,2,0,1,2,0,224,0,65.00,0,0,...,False,False,False,False,False,False,False,True,False,False
1,2,0,2,3,0,5,0,106.68,0,1,...,False,False,False,False,False,False,False,False,True,False
2,1,0,2,1,0,1,0,60.00,0,0,...,False,False,False,False,False,False,False,False,False,False
3,2,0,0,2,0,211,0,100.00,0,0,...,False,False,True,False,False,False,False,False,False,False
4,2,0,1,1,0,48,0,94.50,0,0,...,False,True,False,False,False,False,False,False,False,False
5,2,0,0,2,0,346,0,115.00,0,1,...,False,False,False,False,False,False,True,False,False,False
6,2,0,1,3,0,34,0,107.55,0,1,...,False,False,False,False,False,False,False,True,False,False
7,2,0,1,3,0,83,0,105.61,0,1,...,False,False,False,False,False,False,False,False,False,True
8,3,0,0,4,0,121,0,96.90,0,1,...,False,False,False,False,True,False,False,False,False,False
9,2,0,0,5,0,44,0,133.44,0,3,...,False,False,False,False,False,False,False,True,False,False


In [8]:
def train_test_split_and_scale(data):
    y = data["booking_status"]
    x = data.drop("booking_status",axis=1)
    features = list(x.columns)
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20, random_state = 0)
    scaler = MinMaxScaler()
    x_train = scaler.fit_transform(x_train) # scaling is done only on features
    x_test = scaler.transform(x_test)
    return x_train, x_test, y_train, y_test,features,scaler

In [10]:
x_train, x_test, y_train, y_test,features, scaler = train_test_split_and_scale(data)

In [12]:
x_train.shape,x_test.shape

((29020, 38), (7255, 38))

In [14]:
Counter(y_train)

Counter({0: 19536, 1: 9484})

In [16]:
def fit_and_evaluate_model(x_train, x_test, y_train, y_test,class_weight=None):
    lr = LogisticRegression(class_weight=class_weight)
    model = lr.fit(x_train, y_train) # model training
    lr_predict = lr.predict(x_test) # create predicted o/p 0/1
    lr_conf_matrix = confusion_matrix(y_test, lr_predict)
    lr_acc_score = accuracy_score(y_test, lr_predict)
    lr_mcc_score = matthews_corrcoef(y_test, lr_predict)
    lr_auc_score = roc_auc_score(y_test, lr_predict)
    print("confussion matrix")
    print(lr_conf_matrix)
    print("\n")
    print("Accuracy of Logistic Regression:",lr_acc_score*100,'\n')
    print("MCC Score:", lr_mcc_score*100, '\n')
    print("AUC Score:", lr_auc_score*100, '\n')
    print(classification_report(y_test,lr_predict))
    return model

In [18]:
model = fit_and_evaluate_model(x_train, x_test, y_train, y_test)
print("odds ratio", np.exp(model.coef_))

confussion matrix
[[4326  528]
 [ 884 1517]]


Accuracy of Logistic Regression: 80.53756030323915 

MCC Score: 54.70392818252634 

AUC Score: 76.15219039862356 

              precision    recall  f1-score   support

           0       0.83      0.89      0.86      4854
           1       0.74      0.63      0.68      2401

    accuracy                           0.81      7255
   macro avg       0.79      0.76      0.77      7255
weighted avg       0.80      0.81      0.80      7255

odds ratio [[1.75301805e+00 5.11300948e+00 2.64220199e+00 1.86055832e+00
  2.22076546e-01 1.26827609e+03 2.05746869e-01 1.05482846e+04
  8.01282023e-01 6.37241256e-04 3.91261001e-01 4.36325699e-01
  1.12550082e+00 5.03587720e-01 1.14858613e+00 8.84907222e-01
  9.51189343e-01 9.33965220e-01 5.59133737e-01 5.03741639e-01
  3.80465923e-01 1.95790974e+00 1.47826939e-01 6.72693206e-01
  2.80449253e-01 1.77206938e+00 1.52326160e-01 1.90804956e+00
  1.42571006e+00 1.19063802e+00 8.32496679e-01 1.11119714e+00
  9.

## Improve the model

In [21]:
weight = {0:1, 1:4}
model = fit_and_evaluate_model(x_train, x_test, y_train, y_test,class_weight=weight)

confussion matrix
[[3241 1613]
 [ 267 2134]]


Accuracy of Logistic Regression: 74.0868366643694 

MCC Score: 52.40033883163433 

AUC Score: 77.82465399065455 

              precision    recall  f1-score   support

           0       0.92      0.67      0.78      4854
           1       0.57      0.89      0.69      2401

    accuracy                           0.74      7255
   macro avg       0.75      0.78      0.73      7255
weighted avg       0.81      0.74      0.75      7255



In [23]:
import joblib
joblib.dump(
    {
        "model": model,
        "scaler": scaler,
        "features": features
    },
    "/home/cloud/ML_Models/logistic_regression.pkl"
)

['/home/cloud/ML_Models/logistic_regression.pkl']